In [ ]:
# Instalacion de dependencias
!pip install pyspark

In [ ]:
# Importacion de dependencias
from pyspark.ml.feature import Tokenizer
from pyspark.sql import SparkSession
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.types import ArrayType
from pyspark.sql.types import StringType
from pyspark.sql.functions import *
import re

In [ ]:
# Creacion del Spark Session
spark = SparkSession.builder.appName('TokenizacionSpark').getOrCreate()
sc = spark.sparkContext

In [ ]:
# Punto 1
## Se busca construir el BoW mas optimizado del archivo solicitado
## Eliminando stop words, tokenizando, haciendo stemming y lemmatizacion

# Lectura del archivo
file = spark.read.text('./data/0704.3504.pdf.txt')

In [ ]:
# Tokenizacion
tokenization = Tokenizer(inputCol='value', outputCol='tokens')

file = file.select(trim(col('value')).alias('value'))

tokens = tokenization.transform(file)

In [ ]:
# Eliminacion de caracteres no alfabeticos
def noAlphaRemove_udf(x):
    newTokens = [re.sub(r'[^A-Za-z0-9]+','',w) for w in x]
    newTokens = [w.lower() for w in x if len(w) > 1]
    newTokens = [w for w in x if w.isalpha()]
    return newTokens
noAlphaRemove = udf(lambda s: noAlphaRemove_udf(s), ArrayType(StringType()))

tokens = tokens.select(noAlphaRemove(col('tokens')).alias('tokens'))
tokens = tokens.where(size(col('tokens'))>0)

In [ ]:
# Remocion de stopwords
stopWordRemover=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')
tokens = stopWordRemover.transform(tokens)
tokens = tokens.select(col('refined_tokens').alias('tokens'))

In [ ]:
# Distribution
tokens = tokens.withColumn('token_count', size(col('tokens')))
tokens.orderBy(rand()).show(10)